In [2]:
import pandas as pd
%matplotlib inline

In [3]:
csv_file = 'taxi.csv.bz2'


In [4]:
!ls -lh $csv_file

-rw-r--r-- 1 miki miki 9.3M Jan  5 14:09 taxi.csv.bz2


In [5]:
!bzcat $csv_file | wc -l

1000002


In [6]:
!bzcat $csv_file | head -5

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount

1,2018-11-01 00:51:36,2018-11-01 00:52:36,1,.00,1,N,145,145,2,2.5,0.5,0.5,0,0,0.3,3.8
1,2018-11-01 00:07:47,2018-11-01 00:21:43,1,2.30,1,N,142,164,1,11,0.5,0.5,2.45,0,0.3,14.75
1,2018-11-01 00:24:27,2018-11-01 00:34:29,1,1.80,1,N,164,48,2,8.5,0.5,0.5,0,0,0.3,9.8

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = taxi.csv.bz2, output file = (stdout)


In [7]:
df = pd.read_csv(csv_file)

In [8]:
df.shape

(1000000, 17)

In [9]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object

In [10]:
time_cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
df = pd.read_csv(csv_file, parse_dates=time_cols)

In [11]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

In [12]:
df.sample(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
361873,2,2018-11-02 08:28:01,2018-11-02 08:34:58,1,1.36,1,N,162,107,1,7.0,0.0,0.5,1.56,0.0,0.3,9.36
583293,2,2018-11-02 21:11:34,2018-11-02 21:14:51,2,0.84,1,N,233,229,2,5.0,0.5,0.5,0.00,0.0,0.3,6.30
164745,1,2018-11-01 15:10:58,2018-11-01 15:27:09,1,1.50,1,N,230,234,2,11.0,0.0,0.5,0.00,0.0,0.3,11.80
58290,1,2018-11-01 08:23:18,2018-11-01 08:53:09,3,6.00,1,N,161,88,1,25.0,0.0,0.5,6.45,0.0,0.3,32.25
272955,2,2018-11-01 21:51:46,2018-11-01 22:07:01,3,2.33,1,N,162,224,2,11.5,0.5,0.5,0.00,0.0,0.3,12.80


In [13]:
df['VendorID'].unique()

array([1, 2, 4])

In [14]:
vendors = {
    1: 'Creative',
    2: 'VeriFone',
    4: 'BigApple',
}
df['Vendor'] = df['VendorID'].map(vendors)

In [16]:
df['Vendor'].unique()

array(['Creative', 'VeriFone', 'BigApple'], dtype=object)

In [17]:
# How many rides we have per Vendor?

In [19]:
df['Vendor'].value_counts()

VeriFone    581859
Creative    401516
BigApple     16625
Name: Vendor, dtype: int64

In [20]:
# What is the median trip duration in minutes?

In [21]:
row = df.iloc[17]
row['tpep_dropoff_datetime'] - row['tpep_pickup_datetime']

Timedelta('0 days 00:21:23')

In [22]:
durations = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
durations.max(), durations.min()

(Timedelta('0 days 23:59:54'), Timedelta('-5 days +00:19:24'))

In [25]:
md = durations.median()
md.total_seconds() / 60

11.95

In [26]:
import numpy as np
md / np.timedelta64(1, 'm')

11.95

In [27]:
md / np.timedelta64(1, 'h')

0.19916666666666666

In [28]:
# How many rides between 11am to 12pm had more than 1 passanger?

In [35]:
mask = (df['tpep_pickup_datetime'].dt.hour == 11) & (df['passenger_count'] > 1)
len(df[mask])

11788